In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

profs = pd.read_csv("docentes.csv",sep=";")

matriculas = pd.read_csv("matricula-componente-20172.csv",sep=";")

turmas = pd.read_csv("turmas-20172.csv",sep=";",usecols=["id_turma","id_componente_curricular","codigo_turma","id_docente_interno","local","situacao_turma"])
turmas = turmas.dropna(axis=0)

componentes = pd.read_csv("componentes-curriculares-presenciais.csv",sep=";",usecols=["id_componente","codigo","nome","unidade_responsavel"],error_bad_lines=False)
componentes = componentes.dropna(axis=0)

comp_imd = pd.DataFrame()#dataframe responsavel por ter componentes do imd
comp_imd = componentes[componentes['unidade_responsavel'].str.contains("METROPOLE DIGITAL")]
comp_dimap = componentes[componentes['unidade_responsavel'].str.contains("MATEMÁTICA APLICADA")]

frames = [comp_imd,comp_dimap]#frames dos comp do dimap e do imd
comp_imd = pd.concat(frames)#juntando os componentes do dimap e do imd no imd

comp_imd.to_csv("comp_imd.csv",sep=";",index=False)

comp_imd.id_componente = comp_imd.id_componente.astype(int)#para permitir a comparacao com os valores das tabelas...
#nao criadas artificialmente cujos dtypes sao diferentes de Object

#comp_imd = pd.read_csv("componentes-curriculares-imd.csv",sep=";",error_bad_lines=False)

In [63]:
#np.unique(matriculas[['discente']].values)
#componentes[componentes['codigo'].str.contains('IMD',na=False)]


yyy = turmas.dropna(axis=0)

yyy = yyy[yyy['local'].str.contains("CIVT")]

yyy['id_docente_interno']= yyy['id_docente_interno'].astype(int)

#adicionando tabela de professores a tabela de turmas
yyy['prof_nome'] = 'nope'
yyy['tempo_servico'] = 'nope'
for index,row in yyy.iterrows():
    for id2, r2 in profs.iterrows():
        if row['id_docente_interno'] == r2['id_servidor']:
            yyy.loc[index,'prof_nome'] = r2['nome']
            yyy.loc[index,'tempo_servico'] = r2['admissao']
            
#adicionando tabela de componentes do imd/dimap na tabela de turmas            
yyy['nome_materia'] = 'nope'
for idx1,row1 in yyy.iterrows():
    for idx2, row2 in comp_imd.iterrows():
        #if str(row1['id_componente_curricular']) == str(row2['id_componente']):#funciona em caso de probl. de comparacao
        if row1['id_componente_curricular'] == row2['id_componente']:
            yyy.loc[idx1,'nome_materia'] = row2['nome']
            

#yyy = yyy[yyy['situacao_turma'].str.contains("CONSOLIDADA")]#caso queira pegar so as turmas consolidadas
yyy.head()

,id_turma,codigo_turma,id_docente_interno,id_componente_curricular,local,situacao_turma,prof_nome,tempo_servico,nome_materia
6920,57606915,01,5752467,2022854,A303 - CIVT,CONSOLIDADA,EDGARD DE FARIA CORREA,2009-03-27 00:00:00,CIRCUITOS LÓGICOS
6921,57606916,01,5752467,2022855,A303 - CIVT,CONSOLIDADA,EDGARD DE FARIA CORREA,2009-03-27 00:00:00,LABORATÓRIO DE CIRCUITOS LÓGICOS
6928,57606921,01,5754320,40134,B204 - CIVT,CONSOLIDADA,LEONARDO CUNHA DE MIRANDA,2011-07-27 00:00:00,INTERACAO HUMANO-COMPUTADOR
6930,57606923,01,62353,2040950,A303 - CIVT,CONSOLIDADA,SELAN RODRIGUES DOS SANTOS,2006-01-09 00:00:00,COMPUTAÇÃO GRÁFICA I
6932,57606925,01,5758443,2054904,A304 - CIVT,CONSOLIDADA,EVERTON RANIELLY DE SOUSA CAVALCANTE,2016-06-10 00:00:00,BOAS PRÁTICAS DE PROGRAMAÇÃO


In [7]:
#TESTE
#Criar um DataFrame com as matriculas do semestre
matriculas = pd.read_csv("matricula-componente-20172.csv",sep=';', 
                          usecols=["id_turma","descricao","discente", "unidade"])
matriculas = matriculas[['id_turma','descricao','discente']].drop_duplicates()
#Tabela pivot com a id das turmas como indice e o numero de alunos para cada valor de descricão(aprovado, reprovado etc.)
matricula_pivot = matriculas.pivot_table(index=["id_turma"], columns="descricao", values=["discente"], aggfunc="count").fillna(0)
#Adiciona coluna com o numero total de alunos inscritos na turma
matricula_pivot['total'] = matricula_pivot.sum(axis=1)
#Separa apenas aqueles que foram aprovados
index = matricula_pivot.index
aprovados = matricula_pivot.loc[index,(slice(None),['APROVADO', 'APROVADO POR NOTA'])].sum(axis=1)
matricula_pivot['total_aprovados'] = aprovados
#descobre o valor para a taxa de aprovação de cada turma
aprovacao = aprovados / matricula_pivot['total']
matricula_pivot['aprovacao'] = aprovacao
#Manter no dataframe apenas os valores relevantes
matricula_pivot.columns = matricula_pivot.columns.droplevel(1)
#matricula_pivot = matricula_pivot.drop(columns=['discente'])

matricula_pivot = matricula_pivot[['total_aprovados','total','aprovacao']]

#criando uma coluna para os indices da tabela pivo
matricula_pivot['indices'] = "nope"

iterador = 0
for idx,row in matricula_pivot.iterrows():
    matricula_pivot.loc[idx,'indices'] = matricula_pivot.index.values[iterador]
    iterador = iterador + 1
    
matricula_pivot.head()

,total_aprovados,total,aprovacao,indices
id_turma,,,,
57602444,0.0,81.0,0.000000,57602444
57602445,79.0,86.0,0.918605,57602445
57602446,76.0,84.0,0.904762,57602446
57602447,0.0,82.0,0.000000,57602447
57602448,0.0,78.0,0.000000,57602448


In [47]:
#TESTE
#yyy['tempo_servico'].str.split()
'''year_time = yyy.iloc[0]['tempo_servico'].split()
ymd = year_time[0].split('-')
year = ymd[0]
2018 - int(year)'''

#yyy.loc[yyy['prof_nome'] == "EDGARD DE FARIA CORREA"]
#yyy[yyy.duplicated(['prof_nome'])]
#yyy = yyy[yyy['qtd_alunos'] != "nope"]
yyy.qtd_alunos = yyy.qtd_alunos.astype(int)
yyy.aprovados = yyy.aprovados.astype(int)
yyy.perc_aprovacao = yyy.perc_aprovacao.astype(int)
grupo = yyy.groupby('prof_nome')["qtd_alunos","aprovados"].sum()
#yyy.groupby('prof_nome')["perc_aprovacao"].mean()


#grupo = grupo[grupo['qtd_alunos'] != "nope"]
#grupo = grupo[grupo['aprovados'] != "nope"]
grupo = grupo[grupo.index != 0]


iterador = 0
for idx,row in grupo.iterrows():
    grupo.loc[idx,'profs'] = grupo.index.values[iterador]
    iterador = iterador + 1

grupo['profs'] = grupo['profs'].astype('str')
    
grupo.index = range(len(grupo))

grupo.dtypes
#plt.scatter(grupo.index,grupo['qtd_alunos'])

#grupo = grupo.sort_values(by="profs")
#plt.xticks(grupo.profs, rotation=90)


#plt.show()

#grupo

qtd_alunos     int32
aprovados      int32
profs         object
dtype: object

In [50]:
def incremetSemester(x):
    sem = x
    if (x % 10) == 1:
        sem = sem + 1
    else:
        sem = sem + 9
    return sem

In [56]:
#Função para gerar um dataframe com as taxas de aprovação das turmas de um semestre
def aprovacaoTurma(url):
    matriculas = pd.read_csv(url,sep=';', 
                          usecols=["id_turma","descricao","discente", "unidade"])
    matriculas = matriculas[['id_turma','descricao','discente']].drop_duplicates()
    #Tabela pivot com a id das turmas como indice e o numero de alunos para cada valor de descricão(aprovado, reprovado etc.)
    matricula_pivot = matriculas.pivot_table(index=["id_turma"], columns="descricao", values=["discente"], aggfunc="count").fillna(0)
    #Adiciona coluna com o numero total de alunos inscritos na turma
    matricula_pivot['total'] = matricula_pivot.sum(axis=1)
    #Separa apenas aqueles que foram aprovados
    index = matricula_pivot.index
    aprovados = matricula_pivot.loc[index,(slice(None),['APROVADO', 'APROVADO POR NOTA'])].sum(axis=1)
    matricula_pivot['total_aprovados'] = aprovados
    #descobre o valor para a taxa de aprovação de cada turma
    aprovacao = aprovados / matricula_pivot['total']
    matricula_pivot['aprovacao'] = aprovacao
    #Manter no dataframe apenas os valores relevantes
    matricula_pivot.columns = matricula_pivot.columns.droplevel(1)
    #matricula_pivot = matricula_pivot.drop(columns=['discente'])

    matricula_pivot = matricula_pivot[['total_aprovados','total','aprovacao']]

    #criando uma coluna para os indices da tabela pivo
    matricula_pivot['indices'] = "nope"

    iterador = 0
    for idx,row in matricula_pivot.iterrows():
        matricula_pivot.loc[idx,'indices'] = matricula_pivot.index.values[iterador]
        iterador = iterador + 1
    return matricula_pivot

In [60]:
#Criar dataframes para cada um doos datasets utilizados
import datetime as d
base_url = 'matricula-componente-'
base_sem = 20091
dataList = []
now = int(d.datetime.now().year)
iterate = (now - base_sem//10)*2
for x in range(0, iterate):
    url = base_url + str(base_sem) + '.csv'
    dataList.append(aprovacaoTurma(url))
    base_sem = incremetSemester(base_sem)

In [61]:
#Fazer uma cópia dos dados por segurança
data = dataList

In [62]:
#Concatenando todos os dataset em um unico dataframe
totalData = data[0]
for y in range (1,18):
    totalData = pd.concat([totalData,data[y]])
totalData.axes

[Int64Index([ 1151199,  1160109,  1167001,  1167002, 57226951, 57226952,
             57227013, 57483959, 57486803, 57487100,
             ...
             57618473, 57618593, 57618898, 57618988, 57620165, 57620182,
             57620200, 57620243, 57620655, 57621958],
            dtype='int64', name='id_turma', length=140813),
 Index(['total_aprovados', 'total', 'aprovacao', 'indices'], dtype='object')]

In [95]:
yyy['qtd_alunos'] = "nope"
yyy['aprovados'] = "nope"
yyy['perc_aprovacao'] = "nope"

#Função que recebe dois dataframes, os relaciona, e retorna o primeiro dataframe com os valores adicionais do segundo
def mergeData(table1, table2):
    for idx1,row1 in table1.iterrows():
        for idx2,row2 in table2.iterrows():
            if row1['id_turma'] == row2['indices']:
                table1.loc[idx1,'qtd_alunos'] = row2['total']
                table1.loc[idx1,'aprovados'] = row2['total_aprovados']
                table1.loc[idx1,'perc_aprovacao'] = row2['aprovacao']
    return table1

In [99]:
#testando a função
profList.append(mergeData(yyy, data[0]))
#if (len(yyy[yyy.qtd_alunos != "nope"]) > 0):
#    profList.append(yyy)
#    semList.append(base_sem)
#base_sem = incremetSemester(base_sem)

#yyy

[       id_turma codigo_turma  id_docente_interno  id_componente_curricular  \
 6920   57606915           01             5752467                   2022854   
 6921   57606916           01             5752467                   2022855   
 6928   57606921           01             5754320                     40134   
 6930   57606923           01               62353                   2040950   
 6932   57606925           01             5758443                   2054904   
 6933   57606926           01             5754320                   2054906   
 6934   57606927           01             5751828                   2054905   
 6935   57606928           01               22764                   2054912   
 6936   57606929           01             5752649                   2054915   
 6937   57606930           01               22764                   2054917   
 6938   57606930           01               23809                   2054917   
 6941   57606932           01             5752241   

In [105]:
#Itera pelo dataframe das matriculas e o relaciona com o dataframe dos professores do IMD
profList = []
semList = []
base_sem = 20091
for j in range(0, len(data)):
    aux = mergeData(yyy, data[j])
    #if(len(aux[aux.qtd_alunos != "nope"]) > 0):
    profList.append(aux)
    semList.append(base_sem)
    base_sem = incremetSemester(base_sem)
#print(len(profList))
#print(base_sem)
#profList[1]

2
20101


,id_turma,codigo_turma,id_docente_interno,id_componente_curricular,local,situacao_turma,prof_nome,tempo_servico,nome_materia,qtd_alunos,aprovados,perc_aprovacao
6920,57606915,01,5752467,2022854,A303 - CIVT,CONSOLIDADA,EDGARD DE FARIA CORREA,2009-03-27 00:00:00,CIRCUITOS LÓGICOS,33,12,0.363636
6921,57606916,01,5752467,2022855,A303 - CIVT,CONSOLIDADA,EDGARD DE FARIA CORREA,2009-03-27 00:00:00,LABORATÓRIO DE CIRCUITOS LÓGICOS,33,9,0.272727
6928,57606921,01,5754320,40134,B204 - CIVT,CONSOLIDADA,LEONARDO CUNHA DE MIRANDA,2011-07-27 00:00:00,INTERACAO HUMANO-COMPUTADOR,14,14,1
6930,57606923,01,62353,2040950,A303 - CIVT,CONSOLIDADA,SELAN RODRIGUES DOS SANTOS,2006-01-09 00:00:00,COMPUTAÇÃO GRÁFICA I,26,21,0.807692
6932,57606925,01,5758443,2054904,A304 - CIVT,CONSOLIDADA,EVERTON RANIELLY DE SOUSA CAVALCANTE,2016-06-10 00:00:00,BOAS PRÁTICAS DE PROGRAMAÇÃO,45,24,0.533333
6933,57606926,01,5754320,2054906,B204 - CIVT,CONSOLIDADA,LEONARDO CUNHA DE MIRANDA,2011-07-27 00:00:00,ANÁLISE E PROJETO ORIENTADO A OBJETOS,30,20,0.666667
6934,57606927,01,5751828,2054905,A304 - CIVT,CONSOLIDADA,MARCEL VINICIUS MEDEIROS OLIVEIRA,2008-07-14 00:00:00,LÓGICA APLICADA A ENGENHARIA DE SOFTWARE,25,16,0.64
6935,57606928,01,22764,2054912,A305 - CIVT,CONSOLIDADA,JAIR CAVALCANTI LEITE,1993-04-01 00:00:00,PROJETO DE INTERFACES DE USUÁRIO,77,38,0.493506
6936,57606929,01,5752649,2054915,A309 - CIVT,CONSOLIDADA,ROBERTA DE SOUZA COELHO,2009-07-01 00:00:00,TESTE DE SOFTWARE II,10,9,0.9
6937,57606930,01,22764,2054917,A309 - CIVT,CONSOLIDADA,JAIR CAVALCANTI LEITE,1993-04-01 00:00:00,ARQUITETURA DE SOFTWARE,26,19,0.730769


In [ ]:
def datasetMatricula(year):
    matriculas = pd.read_csv(year,sep=";",error_bad_lines=False)
    return matriculas
def datasetTurma(year):
    turmas = pd.read_csv(year,sep=";",error_bad_lines=False)
    return turmas